In [6]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import folium
from folium.plugins import MarkerCluster
import os
import time
import datetime

In [7]:
import pyroutelib3
from pyroutelib3 import Router
import requests, json
import urllib.parse
import tempfile
import zipfile

In [8]:
import tkinter as tk
from tkinter import simpledialog
from tkinter import messagebox
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

# I) Utilisation du GPS 

## Choix du point de départ et du point d'arrivée

In [2]:
adresse_depart =input("D'où partez-vous ? (numéro, rue, code postal ville)")

adresse_arrivee = input("Où voulez-vous aller ? (numéro, rue, code postal ville)")

D'où partez-vous ? (numéro, rue, code postal ville) 5, Avenue Anatole France, 75007 Paris
Où voulez-vous aller ? (numéro, rue, code postal ville) 1, Rue de Rivoli, 75001 Paris


In [3]:
#print(adresse_depart)

5, Avenue Anatole France, 75007 Paris


In [4]:
#print(adresse_arrivee)

1, Rue de Rivoli, 75001 Paris


In [24]:
#adresse_depart = "5, Avenue Anatole France, 75007 Paris"
#adresse_arrivee = "1, Rue de Rivoli, 75001 Paris"
dep_json = requests.get("https://api-adresse.data.gouv.fr/search/?q=" + urllib.parse.quote(adresse_depart) + "&format=json").json()
arr_json = requests.get("https://api-adresse.data.gouv.fr/search/?q=" + urllib.parse.quote(adresse_arrivee) + "&format=json").json()
coord_dep = list(dep_json['features'][0].get('geometry').get('coordinates'))
coord_arr = list(arr_json['features'][0].get('geometry').get('coordinates'))

In [109]:
x = input("Avez-vous votre propre vélo ? Entrez 'oui' ou 'non' ")

Avez-vous votre propre vélo ? Entrez 'oui' ou 'non'  non


## Lorsque l'utilisateur n'a pas son propre vélo

In [116]:
if(x=='non'):
                                                                                 
    coor_depart = [coord_dep[1], coord_dep[0]] 
    coor_arrivee = [coord_arr[1], coord_arr[0]] 
    
    # Ré-initialisation de la carte 
    centre = coor_depart 
    idf = folium.Map(location = centre, zoom_start = 13)
    
    # Calcul de la distance en km entre le point de départ et toutes les stations

    distances = []
    for i in range(len(velos.geometry)):
      a = (max(velos.geometry[i].x, coord_dep[0]) - min(velos.geometry[i].x, coord_dep[0]))**2 + (max(velos.geometry[i].y, coord_dep[1]) - min(velos.geometry[i].y, coord_dep[1]))**2
      a = a**(1/2)*100
      distances.append(a)
    
    #Station la plus proche du point de départ 

    i = distances.index(min(distances))

    station_dep = [velos["geometry"][i].y, velos["geometry"][i].x]

    print( 'Station la plus proche du point de départ :', velos["name"][i] )
    
    # Vérification disponibilité vélo dans la station de départ
    
    if(velos["numbikesavailable"][i]!="0" and velos["numbikesavailable"][i]!="N" ):
        print("\n \n Un vélo est disponible à la station la plus proche du point de départ")
        
    elif(velos["numbikesavailable"][i]=="N"):
        print("\n \n Nous ne disposons pas d'information concernant la disponibilité d'un vélo à la station la plus proche du point de départ")
        
    elif(velos["numbikesavailable"][i]=="0"):
        print("\n \n Pas de vélo disponible à la station la plus proche du point de départ")

        k = i
        
        while( velos["numbikesavailable"][k]=="0"):
            
            # Nouvelle station la plus proche et avec vélo disponible

            distances.pop(k)
            k = distances.index(min(distances))

            station_dep = [velos["geometry"][k].y, velos["geometry"][k].x]

        print( 'Station la plus proche du point de départ avec vélo disponible : \n', velos["name"][k] )
          
    
    print('\n \nDistance à effectuer à pied (point de départ -> station de départ) :', round(min(distances),2), "km")
    print("soit un temps de trajet d'environ :", round(min(distances)*1000/66.67), "minutes\n \n")
    
    t = min(distances)*1000/66.67
        
    
    # Calcul de la distance en km entre le point d'arrivée et toutes les stations

    distances = []
    for i in range(len(velos.geometry)):
      a = (max(velos.geometry[i].x, coord_arr[0]) - min(velos.geometry[i].x, coord_arr[0]))**2 + (max(velos.geometry[i].y, coord_arr[1]) - min(velos.geometry[i].y, coord_arr[1]))**2
      a = a**(1/2)*100
      distances.append(a)
    
    #Station la plus proche du point d'arrivée

    j = distances.index(min(distances))

    station_arr = [velos["geometry"][j].y, velos["geometry"][j].x] 
    
    print( 'Station la plus proche du point d arrivée : ', velos["name"][j] )
    
    # Vérification disponibilité dock dans la station d'arrivée
    
    if(velos["numdocksavailable"][j]!="0" and velos["numdocksavailable"][j]!="N" ):
        print("\n Un dock est disponible à la station la plus proche du point d'arrivée")
    
    elif(velos["numdocksavailable"][j]=="N"):
        print("\nNous ne disposons pas d'information concernant la disponibilité d'un dock à la station la plus proche du point d'arrivée")
    
    elif(velos["numdocksavailable"][j]=="0"):
        print("\nPas de dock disponible à la station la plus proche du point d'arrivée'")
        
        while( velos["numdocksavailable"][j]=="0"):
            
            # Nouvelle station la plus proche et avec dock disponible

            distances.pop(j)
            j = distances.index(min(distances))

            station_arr = [velos["geometry"][j].y, velos["geometry"][j].x]

        print( "Station la plus proche du point d'arrivée avec dock disponible : \n", velos["name"][k] )
        
    # Durée et distance trajet à vélo
    
    d = (max(station_dep[0], station_arr[0]) - min(station_dep[0], station_arr[0]))**2 + (max(station_dep[1], station_arr[1]) - min(station_dep[1], station_arr[1]))**2
    d = d**(1/2)*100                                                                                                 

    print( '\n \nDistance à effectuer en vélo (station de départ -> station d\'arrivée) :', round(d,2), "km")
    print("soit un temps de trajet d'environ :", round(d*1000/250), "minutes\n")
    print( '\n Distance à effectuer à pied (station d\'arrivée -> point d\'arrivée) :', round(min(distances),2), "km")
    print("soit un temps de trajet d'environ :", round(min(distances)*1000/66.67), "minutes\n \n")
    
    print("Le temps total du trajet est donc de :", round(t+d*1000/250+min(distances)*1000/66.67)  ,"minutes\n \n")
    
    # Si le temps total du trajet calculé > temps de trajet en faisant tout à pied, alors faire le trajet à pied 
    
    d_pied = (max(coord_dep[0], coord_arr[0]) - min(coord_dep[0], coord_arr[0]))**2 + (max(coord_dep[1], coord_arr[1]) - min(coord_dep[1], coord_arr[1]))**2
    d_pied = d_pied**(1/2)*100
    t_pied = d_pied*1000/66.67
    
    if( t+d*1000/250+min(distances)*1000/66.67 > t_pied ):
        
        print("ATTENTION : Il serait plus judicieux de faire tout le trajet à pied car :")
        print("Distance du point de départ au point d'arrivée :", round(d_pied,2), "km")
        print("Temps du trajet à pied :", round(t_pied), "minutes\n" )
            
    #Trajet à pied : point de départ -> station départ

    folium.Marker(coor_depart,popup="Depart").add_to(idf)
    folium.Marker(coor_arrivee,popup="Arrivee").add_to(idf)
    
    router = pyroutelib3.Router("foot")

    depart = router.findNode(coor_depart[0], coor_depart[1])
    arrivee = router.findNode(station_dep[0], station_dep[1])

    routeLatLons=[coor_depart,station_dep]
    status, route = router.doRoute(depart, arrivee)
    if status == 'success':
      #print("votre trajet existe")
      routeLatLons = list(map(router.nodeLatLon, route))
    else:
      print("Votre trajet n'existe pas")

    folium.PolyLine(routeLatLons, color="black", weight=2.5, opacity=1).add_to(idf)
    
    # Affichage des lieux accidentogènes situés à moins de 50 mètres du trajet 

    centroids = pd.read_csv("centroids_df.csv")

    dist_centroids = []
    centroids_50 = []

    for i in range( centroids.shape[0]) : 
        for k in routeLatLons : 
            dist = (k[0]- centroids["lat"][i] )**2 + (k[1]- centroids["lon"][i] )**2
            dist = dist**1/2*100
            if dist < 0.0005 :
                dist_centroids.append(dist)
                centroids_50.append([centroids["lat"][i], centroids["lon"][i]])

    for i in range(len(centroids_50)) : 
        coord = centroids_50[i]
        folium.Circle(coord,radius=60, popup='Lieu accidentogene',color='crimson',fill=True,).add_to(idf)
    
    # Trajet en vélo : station départ -> station arrivée
    
    router = pyroutelib3.Router("cycle")
    depart = router.findNode(station_dep[0], station_dep[1])
    arrivee = router.findNode(station_arr[0], station_arr[1])

    routeLatLons=[station_dep,station_arr]
    status, route = router.doRoute(depart, arrivee)
    if status == 'success':
      #print("votre trajet existe")
      routeLatLons = list(map(router.nodeLatLon, route))
    else:
      print("votre trajet n'existe pas")

    folium.PolyLine(routeLatLons, color="blue", weight=2.5, opacity=1).add_to(idf)
    
     # Affichage des lieux accidentogènes situés à moins de 50 mètres du trajet 

    centroids = pd.read_csv("centroids_df.csv")

    dist_centroids = []
    centroids_50 = []

    for i in range( centroids.shape[0]) : 
        for k in routeLatLons : 
            dist = (k[0]- centroids["lat"][i] )**2 + (k[1]- centroids["lon"][i] )**2
            dist = dist**1/2*100
            if dist < 0.0005 :
                dist_centroids.append(dist)
                centroids_50.append([centroids["lat"][i], centroids["lon"][i]])

    for i in range(len(centroids_50)) : 
        coord = centroids_50[i]
        folium.Circle(coord,radius=60, popup='Lieu accidentogene',color='crimson',fill=True,).add_to(idf)
        
    # Trajet à pied : station arrivée -> point d'arrivée
    
    router = pyroutelib3.Router("foot")
    depart = router.findNode(station_arr[0], station_arr[1])
    arrivee = router.findNode(coor_arrivee[0], coor_arrivee[1])

    routeLatLons=[station_arr, coor_arrivee]
    status, route = router.doRoute(depart, arrivee)
    if status == 'success':
      #print("votre trajet existe")
      routeLatLons = list(map(router.nodeLatLon, route))
    else:
      print("Votre trajet n'existe pas")

    folium.PolyLine(routeLatLons, color="black", weight=2.5, opacity=1).add_to(idf)
    
     # Affichage des lieux accidentogènes situés à moins de 50 mètres du trajet 

    centroids = pd.read_csv("centroids_df.csv")

    dist_centroids = []
    centroids_50 = []

    for i in range( centroids.shape[0]) : 
        for k in routeLatLons : 
            dist = (k[0]- centroids["lat"][i] )**2 + (k[1]- centroids["lon"][i] )**2
            dist = dist**1/2*100
            if dist < 0.0005 :
                dist_centroids.append(dist)
                centroids_50.append([centroids["lat"][i], centroids["lon"][i]])

    for i in range(len(centroids_50)) : 
        coord = centroids_50[i]
        folium.Circle(coord,radius=60, popup='Lieu accidentogene',color='crimson',fill=True,).add_to(idf)
    
    

Station la plus proche du point de départ : Bourdonnais - Tour Eiffel

 
 Un vélo est disponible à la station la plus proche du point de départ

 
Distance à effectuer à pied (point de départ -> station de départ) : 0.23 km
soit un temps de trajet d'environ : 3 minutes
 

Station la plus proche du point d arrivée :  Saint-Honoré - Musée du Louvre

 Un dock est disponible à la station la plus proche du point d'arrivée

 
Distance à effectuer en vélo (station de départ -> station d'arrivée) : 3.94 km
soit un temps de trajet d'environ : 16 minutes


 Distance à effectuer à pied (station d'arrivée -> point d'arrivée) : 0.13 km
soit un temps de trajet d'environ : 2 minutes
 

Le temps total du trajet est donc de : 21 minutes
 



In [117]:
idf

# Lorsque l'utilisateur a son propre vélo

In [107]:
if(x=='oui'):
    
    coor_depart = [coord_dep[1], coord_dep[0]] 
    coor_arrivee = [coord_arr[1], coord_arr[0]] 
    
    # Ré-initialisation de la carte 
    centre = coor_depart 
    idf = folium.Map(location = centre, zoom_start = 13)

    folium.Marker(coor_depart,popup="Depart").add_to(idf)
    folium.Marker(coor_arrivee,popup="Arrivee").add_to(idf)
    
    # Trajet en vélo : point départ -> point d'arrivée
    
    router = pyroutelib3.Router("cycle")
    depart = router.findNode(coor_depart[0], coor_depart[1])
    #print(depart)
    arrivee = router.findNode(coor_arrivee[0], coor_arrivee[1])
    #print(arrivee)

    routeLatLons=[coor_depart,coor_arrivee]
    status, route = router.doRoute(depart, arrivee)
    if status == 'success':
      #print("Votre trajet existe")
      routeLatLons = list(map(router.nodeLatLon, route))
    else:
      print("Votre trajet n'existe pas")

    folium.PolyLine(routeLatLons, color="blue", weight=2.5, opacity=1).add_to(idf)
    
    # Affichage des clusters situés à moins de 50 mètres du trajet 

    centroids = pd.read_csv("centroids_df.csv")

    dist_centroids = []
    centroids_50 = []

    for i in range( centroids.shape[0]) : 
        for k in routeLatLons : 
            dist = (k[0]- centroids["lat"][i] )**2 + (k[1]- centroids["lon"][i] )**2
            dist = dist**1/2*100
            if dist < 0.0005 :
                dist_centroids.append(dist)
                centroids_50.append([centroids["lat"][i], centroids["lon"][i]])

    for i in range(len(centroids_50)) : 
        coord = centroids_50[i]
        folium.Circle(coord,radius=60, popup='Lieu accidentogene',color='crimson',fill=True,).add_to(idf)
        


In [108]:
idf